In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 1. 행정구별 공원 갯수

### 1-1. 파일 불러오기

In [2]:
# 상대 경로 설정
park_file = '../6.Infrastructure/공원좌표.csv'
boundary_file = '../4.Public_transportation/LARD_ADM_SECT_SGG_gu/LARD_ADM_SECT_SGG_11_202405.shp'

# 데이터 불러오기
gdf = gpd.read_file(boundary_file)
park_df = pd.read_csv(park_file, encoding="CP949")

### 1-2. 공원 데이터를 GeoDataFrame으로 변환

In [3]:
# 공원 데이터를 GeoDataFrame으로 변환 (WGS84 좌표계)
park_geometry = gpd.points_from_xy(park_df['X좌표(WGS84)'], park_df['Y좌표(WGS84)'])
park_gdf = gpd.GeoDataFrame(park_df, geometry=park_geometry, crs='EPSG:4326')  # WGS84 좌표계

### 1-3. 공원 데이터 좌표계를 행정구 데이터 좌표계로 변환

In [4]:
# 공원 데이터 좌표계를 행정구 데이터 좌표계로 변환
park_gdf = park_gdf.to_crs(gdf.crs)

### 1-4. 공원과 행정구 경계 데이터 결합

In [5]:
# 공원과 행정구 경계 데이터 결합
joined_gdf = gpd.sjoin(park_gdf, gdf, how='inner', predicate='within')

### 1-5. 행정구별 공원 개수 계산 및 결과 출력

In [6]:
# 행정구별 공원 개수 계산
park_count = joined_gdf.groupby('SGG_NM').size().reset_index(name='공원_개수')

# 결과 출력
print(park_count)

        SGG_NM  공원_개수
0    서울특별시 강남구      5
1    서울특별시 강동구      6
2    서울특별시 강북구      3
3    서울특별시 강서구      9
4    서울특별시 관악구      2
5    서울특별시 광진구      2
6    서울특별시 구로구      4
7    서울특별시 금천구      4
8    서울특별시 도봉구      6
9   서울특별시 동대문구      4
10   서울특별시 동작구      6
11   서울특별시 마포구      4
12  서울특별시 서대문구      4
13   서울특별시 서초구      4
14   서울특별시 성동구      5
15   서울특별시 성북구      4
16   서울특별시 송파구      7
17   서울특별시 양천구      5
18  서울특별시 영등포구      5
19   서울특별시 용산구      3
20   서울특별시 은평구      5
21   서울특별시 종로구     11
22    서울특별시 중구      2
23   서울특별시 중랑구      3


### 1-6. 결과 저장

In [11]:
# 결과 저장
# UTF-8-SIG로 저장
park_count.to_csv('Data_Preprocessing/6_Park_count_by_district.csv', 
                  index=False, encoding='utf-8-sig')